# Experiment 2
This eperiment aims to combine both efforts of:
- The Knowledge graph used to store the ontology data
- The LNN model used to establish rules on the data relationships

## Questions that need answering:
1. How to form the interaction between the graph and the model? i.e., where does the graph stop and the model start? (*architecture*)
2. How to write the model rules? Does the output of the rule create a node object for the graph/a query object for the graph/a model object in itself/a rule for the model?

In [1]:
import kglab

# create a KnowledgeGraph object
kg = kglab.KnowledgeGraph()

# load RDF from a URL
kg.load_rdf("..\data\graph database\protontop.rdf", format="xml")

# measure the graph
measure = kglab.Measure()
measure.measure_graph(kg)

print("edges: {}\n".format(measure.get_edge_count()))
print("nodes: {}\n".format(measure.get_node_count()))

edges: 482

nodes: 114



In [2]:
# get graph triples
for s, p, o in kg.rdf_graph():
    print(s, p, o)

http://www.ontotext.com/proton/protontop#validUntil http://www.w3.org/2000/01/rdf-schema#label Valid until
http://www.ontotext.com/proton/protontop#Role http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.ontotext.com/proton/protontop#Situation
http://www.ontotext.com/proton/protontop#hasContactInfo http://www.w3.org/2000/01/rdf-schema#domain http://www.ontotext.com/proton/protontop#Object
http://www.ontotext.com/proton/protontop#hasSubject http://www.w3.org/2000/01/rdf-schema#label has Subject
http://www.ontotext.com/proton/protontop#hasDate http://www.w3.org/2000/01/rdf-schema#domain http://www.ontotext.com/proton/protontop#InformationResource
http://www.ontotext.com/proton/protontop#partOf http://www.w3.org/2000/01/rdf-schema#range http://www.ontotext.com/proton/protontop#Entity
http://www.ontotext.com/proton/protontop#latitude http://www.w3.org/2000/01/rdf-schema#domain http://www.ontotext.com/proton/protontop#Location
http://www.ontotext.com/proton/protontop#doingBusinessAs

In [3]:
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig06.html")

tmp.fig06.html


In [4]:
# load query results in dataframe
import pandas as pd

df = kg.query_as_df(sparql)
df

,subject,label,supertype
0,ptop:Abstract,Abstract,ptop:Entity
1,ptop:Agent,Agent,ptop:Object
2,ptop:ContactInformation,Contact Information,ptop:Abstract
3,ptop:Document,Document,ptop:InformationResource
4,ptop:Entity,Entity,NaN
5,ptop:Event,Event,ptop:Happening
6,ptop:GeneralTerm,General Term,ptop:Abstract
7,ptop:Group,Group,ptop:Agent
8,ptop:Happening,Happening,ptop:Entity
9,ptop:InformationResource,Information Resource,ptop:Statement


In [5]:
# make sure the language concept class exists for the PROTON ontology
language_proton_df = df[df["label"] == "Language"]
language_proton_df

,subject,label,supertype
11,ptop:Language,Language,ptop:Abstract


In [6]:
# query filtered class hierarchy
sparql_2 = """
SELECT *
WHERE {
    ?subject ?property ?value
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph_2 = kg.visualize_query(sparql_2, notebook=True)

pyvis_graph_2.force_atlas_2based()
pyvis_graph_2.show("tmp.fig07.html")

tmp.fig07.html


In [7]:
df_2 = kg.query_as_df(sparql_2)
df_2

,property,supertype,value,label,subject
0,rdfs:subClassOf,ptop:Entity,ptop:Entity,Abstract,ptop:Abstract
1,rdfs:label,ptop:Entity,Abstract,Abstract,ptop:Abstract
2,rdf:type,ptop:Entity,owl:Class,Abstract,ptop:Abstract
3,rdfs:comment,ptop:Entity,"Something, which neither happens nor exists, e...",Abstract,ptop:Abstract
4,rdfs:subClassOf,ptop:Object,ptop:Object,Agent,ptop:Agent
...,...,...,...,...,...
94,rdfs:subClassOf,ptop:Happening,ptop:Happening,Time Interval,ptop:TimeInterval
95,rdfs:subClassOf,ptop:Abstract,ptop:Abstract,Topic,ptop:Topic
96,rdfs:label,ptop:Abstract,Topic,Topic,ptop:Topic
97,rdf:type,ptop:Abstract,owl:Class,Topic,ptop:Topic


In [8]:
df_2_object_properties = df_2[df_2["property"] == "rdfs:subClassOf"]
df_2_object_properties

,property,supertype,value,label,subject
0,rdfs:subClassOf,ptop:Entity,ptop:Entity,Abstract,ptop:Abstract
4,rdfs:subClassOf,ptop:Object,ptop:Object,Agent,ptop:Agent
11,rdfs:subClassOf,ptop:Abstract,ptop:Abstract,Contact Information,ptop:ContactInformation
14,rdfs:subClassOf,ptop:InformationResource,ptop:InformationResource,Document,ptop:Document
22,rdfs:subClassOf,ptop:Happening,ptop:Happening,Event,ptop:Event
24,rdfs:subClassOf,ptop:Abstract,ptop:Abstract,General Term,ptop:GeneralTerm
30,rdfs:subClassOf,ptop:Agent,ptop:Agent,Group,ptop:Group
31,rdfs:subClassOf,ptop:Entity,ptop:Entity,Happening,ptop:Happening
37,rdfs:subClassOf,ptop:Statement,ptop:Statement,Information Resource,ptop:InformationResource
42,rdfs:subClassOf,ptop:SocialPosition,ptop:SocialPosition,Job Position,ptop:JobPosition


In [9]:
# query all properties
sparql_3 = """
SELECT *
  WHERE {
    ?property ?range ?domain
  } ORDER BY ?property
  """

# visualize query
pyvis_graph_3 = kg.visualize_query(sparql_3, notebook=True)

pyvis_graph_3.force_atlas_2based()
pyvis_graph_3.show("tmp.fig08.html")

tmp.fig08.html


In [10]:
df_3 = kg.query_as_df(sparql_3)
df_3

,property,domain,range
0,<http://www.ontotext.com/proton/protontop>,Proton,dc:title
1,<http://www.ontotext.com/proton/protontop>,Proton Ontology,rdfs:label
2,<http://www.ontotext.com/proton/protontop>,Upper-level ontology with extensions to handle...,dc:subject
3,<http://www.ontotext.com/proton/protontop>,"Ontotext (Kiril Simov, Atanas Kiryakov, Ivan T...",dc:creator
4,<http://www.ontotext.com/proton/protontop>,"PROTON (Proto Ontology), Top module",rdfs:comment
...,...,...,...
477,ptop:withinOrganization,Determines in which organization is the position,rdfs:comment
478,ptop:withinOrganization,owl:ObjectProperty,rdf:type
479,rdfs:comment,owl:AnnotationProperty,rdf:type
480,rdfs:label,owl:AnnotationProperty,rdf:type


In [11]:
df_3_object_properties = df_3[df_3["domain"] == "owl:ObjectProperty"]
df_3_object_properties

,property,domain,range
110,ptop:controls,owl:ObjectProperty,rdf:type
112,ptop:derivedFromSource,owl:ObjectProperty,rdf:type
127,ptop:documentAuthor,owl:ObjectProperty,rdf:type
145,ptop:establishedIn,owl:ObjectProperty,rdf:type
165,ptop:hasChild,owl:ObjectProperty,rdf:type
171,ptop:hasContactInfo,owl:ObjectProperty,rdf:type
172,ptop:hasContributor,owl:ObjectProperty,rdf:type
178,ptop:hasCreator,owl:ObjectProperty,rdf:type
187,ptop:hasEmployee,owl:ObjectProperty,rdf:type
191,ptop:hasLeader,owl:ObjectProperty,rdf:type


In [12]:
# query filtered class hierarchy
sparql_4 = """
select *
WHERE {
    ?subject ?property ?value
    {?property a rdf:Property.} UNION {?property a owl:ObjectProperty.} UNION {?property a owl:DatatypeProperty.} UNION {?property a owl:AnnotationProperty.} .
    { ?subject a owl:Class . } UNION { ?subject a owl:ObjectProperty . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label } .
    OPTIONAL { ?property ?range ?domain } .
    OPTIONAL { ?subject rdf:type ?type }
}
ORDER BY ?subject
"""

# visualize query
pyvis_graph_4 = kg.visualize_query(sparql_4, notebook=True)

pyvis_graph_4.force_atlas_2based()
pyvis_graph_4.show("tmp.fig07.html")

tmp.fig07.html


In [13]:
df_4 = kg.query_as_df(sparql_4)
df_4

,property,domain,supertype,type,value,label,range,subject
0,rdfs:label,owl:AnnotationProperty,ptop:Entity,owl:Class,Abstract,Abstract,rdf:type,ptop:Abstract
1,rdfs:comment,owl:AnnotationProperty,ptop:Entity,owl:Class,"Something, which neither happens nor exists, e...",Abstract,rdf:type,ptop:Abstract
2,rdfs:comment,owl:AnnotationProperty,ptop:Object,owl:Class,"An Agent is something, which can show (carry o...",Agent,rdf:type,ptop:Agent
3,rdfs:label,owl:AnnotationProperty,ptop:Object,owl:Class,Agent,Agent,rdf:type,ptop:Agent
4,rdfs:label,owl:AnnotationProperty,ptop:Abstract,owl:Class,Contact Information,Contact Information,rdf:type,ptop:ContactInformation
...,...,...,...,...,...,...,...,...
148,rdfs:label,owl:AnnotationProperty,NaN,owl:TransitiveProperty,Subtopic of,Subtopic of,rdf:type,ptop:subTopicOf
149,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,Subsidiary Organization of,Subsidiary Organization of,rdf:type,ptop:subsidiaryOrganizationOf
150,rdfs:comment,owl:AnnotationProperty,NaN,owl:ObjectProperty,Indicates that an organization is in some sens...,Subsidiary Organization of,rdf:type,ptop:subsidiaryOrganizationOf
151,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,within Organization,within Organization,rdf:type,ptop:withinOrganization


In [14]:
df_4_object_properties = df_4[df_4["type"] == "owl:ObjectProperty"]
df_4_object_properties

,property,domain,supertype,type,value,label,range,subject
50,rdfs:comment,owl:AnnotationProperty,NaN,owl:ObjectProperty,Indicates that an Agent has full control over ...,Controls,rdf:type,ptop:controls
51,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,Controls,Controls,rdf:type,ptop:controls
52,rdfs:comment,owl:AnnotationProperty,NaN,owl:ObjectProperty,A reference to a resource from which the prese...,Derived from Source,rdf:type,ptop:derivedFromSource
53,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,Derived from Source,Derived from Source,rdf:type,ptop:derivedFromSource
54,rdfs:comment,owl:AnnotationProperty,NaN,owl:ObjectProperty,The creator (or one of the creators) of a docu...,Document Author,rdf:type,ptop:documentAuthor
...,...,...,...,...,...,...,...,...
147,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,Subtopic of,Subtopic of,rdf:type,ptop:subTopicOf
149,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,Subsidiary Organization of,Subsidiary Organization of,rdf:type,ptop:subsidiaryOrganizationOf
150,rdfs:comment,owl:AnnotationProperty,NaN,owl:ObjectProperty,Indicates that an organization is in some sens...,Subsidiary Organization of,rdf:type,ptop:subsidiaryOrganizationOf
151,rdfs:label,owl:AnnotationProperty,NaN,owl:ObjectProperty,within Organization,within Organization,rdf:type,ptop:withinOrganization


In [15]:
# visualize graph
VIS_STYLE = {
    "wtm": {
        "color": "orange",
        "size": 40,
    },
    "ind":{
        "color": "blue",
        "size": 30,
    },
}

subgraph = kglab.SubgraphTensor(kg)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig03.html")

tmp.fig03.html


In [16]:
import networkx as nx
sparql='''
select * where {?subject ?relation ?object} order by ?subject
'''
subgraph = kglab.SubgraphMatrix(kg, sparql)
nx_graph = subgraph.build_nx_graph(nx.DiGraph(), bipartite=True)

In [17]:
# list all graph nodes
for nodeid in list(nx_graph.nodes):
    print(nx_graph.nodes[nodeid])

{'label': '<http://www.ontotext.com/proton/protontop>', 'bipartite': 0}
{'label': 'Proton', 'bipartite': 1}
{'label': 'Proton Ontology', 'bipartite': 1}
{'label': 'Upper-level ontology with extensions to handle Linked Open Data (LOD)', 'bipartite': 1}
{'label': 'Ontotext (Kiril Simov, Atanas Kiryakov, Ivan Terziev, Dimitar Manov, Mariana Damova, Svetoslav Petrov)', 'bipartite': 1}
{'label': 'PROTON (Proto Ontology), Top module', 'bipartite': 1}
{'label': 'owl:Ontology', 'bipartite': 1}
{'label': '3.0', 'bipartite': 1}
{'label': 'Proton ontology is available under the Creative Commons Attribution 3.0 Unported license; see http://creativecommons.org/licenses/by/3.0/. In a nutshell, you are free to copy, distribute and transmit the work; to remix/adapt the work (e.g. to import the ontology and create specializations of its elements), as long as you attribute the work in the manner specified by the author or licensor (but not in any way that suggests that they endorse you or your use of th

In [18]:
# now experiment on SUMO ontology
# to see how to treat similar classes
# between different ontologies

# create a KnowledgeGraph object
sumo_kg = kglab.KnowledgeGraph()

# load RDF from a URL
sumo_kg.load_rdf("..\data\graph database\sumo.rdf", format="xml")

# measure the graph
measure = kglab.Measure()
measure.measure_graph(sumo_kg)

print("edges: {}\n".format(measure.get_edge_count()))
print("nodes: {}\n".format(measure.get_node_count()))

edges: 587998

nodes: 258063



In [19]:
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = sumo_kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig16.html")

tmp.fig16.html


In [20]:
sumo_df = sumo_kg.query_as_df(sparql)
sumo_df

,subject,label,supertype
0,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,antarctic area,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
1,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,brandy,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
2,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,celsius degree,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
3,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,centimeter,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
4,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,NaN,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
...,...,...,...
5459,owl:ObjectProperty,NaN,NaN
5460,owl:Ontology,NaN,NaN
5461,owl:SymmetricProperty,NaN,NaN
5462,owl:Thing,NaN,NaN


In [21]:
# make sure the language concept class exists in SUMO
language_sumo_df = sumo_df[sumo_df["label"] == "language"]
language_sumo_df

,subject,label,supertype
2655,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,language,<file:///C:/Users/Hanna/Desktop/AutonomousLear...


In [22]:
# try to merge the Proton ontology
# and the SUMO ontology
# by adding the SUMO ontology to the existing PROTON ontology

kg.load_rdf("..\data\graph database\sumo.ttl", format="ttl")

In [23]:
# now query again to see if we retrive both language concept classes in our unified knowledge graph
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig26.html")

tmp.fig26.html


In [24]:
df = kg.query_as_df(sparql)
df

,subject,label,supertype
0,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,antarctic area,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
1,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,brandy,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
2,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,celsius degree,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
3,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,centimeter,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
4,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,NaN,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
...,...,...,...
5486,owl:ObjectProperty,NaN,NaN
5487,owl:Ontology,NaN,NaN
5488,owl:SymmetricProperty,NaN,NaN
5489,owl:Thing,NaN,NaN


In [25]:
# check if the language concept class exists in unified PROTON + SUMO knowledge graph
language_df = df[df["label"].str.lower() == "language"]
language_df

,subject,label,supertype
2655,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,language,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
5468,ptop:Language,Language,ptop:Abstract


In [26]:
# let's check if "english" exists anywhere
# before trying to re-create the experiment
# and adding the "english" concept in the "language" concept
english_df = df[df["label"].str.lower() == "english"]
english_df

,subject,label,supertype


In [27]:
# "english" doesn't exist
# so now we can try to merge
# the model and the graph
# to see what's the best way
# to make the system learn

# Create FOL knowledge for ontology terms
from lnn import (Predicate, Variable, Join, And,
                 Exists, Equivalent, Implies, ForAll, Model, Fact, World)

model = Model()  # Instantiate a model
x, y, z, w = map(Variable, ['x', 'y', 'z', 'w'])

# Define and add predicates to the model
instance_node = Predicate('Instance') # English is an instance (predicate to represent instance nodes as manifestations)
class_node = Predicate('Class')   # Language is a class (predicate to represent class nodes as concepts)
instanceOf_relationship = Predicate('instanceOf', 2) # English is an instance of Language (predicate to represent instanceOf relationship)
subclassOf_relationship = Predicate('subclassOf', 2) # Language is a subclass of Abstract (predicate to represent subClassOf relationship)
property_node = Predicate('Property') # Title is a Property (predicate to represent property nodes as concepts)
propertyOf_relationship = Predicate('propertyOf', 2) # Title is a property of Information Resource (predicate to represent propertyOf relationship)
subPropertyOf_relationship = Predicate('subPropertyOf', 2) # Title is a subproperty of Name of any kind (predicate to represent subPropertyOF relationship)
inverseOf_relationship = Predicate('inverseOf', 2) # Parent is an inverse of Child (predicate to represent inverseOf relationship)

# Add predicates and rules to the model
model.add_knowledge(instance_node, class_node, instanceOf_relationship, subclassOf_relationship, property_node, propertyOf_relationship, subPropertyOf_relationship, inverseOf_relationship)

In [28]:
# Axioms declarations
axioms = [
    ForAll(x, y, z, Implies(And(instance_node(x), class_node(y), class_node(z), instanceOf_relationship(x, y), subclassOf_relationship(y, z)), instanceOf_relationship(x, z)), name="axiom-propagate-class-instance-to-superclass"),
    ForAll(x, y, z, Implies(And(instance_node(x), class_node(y), property_node(z), instanceOf_relationship(x, y), propertyOf_relationship(z, y)), propertyOf_relationship(z, x)), name="axiom-propagate-class-property-to-instance"),
    ForAll(x, y, z, Implies(And(subPropertyOf_relationship(x, y), propertyOf_relationship(y, z)), propertyOf_relationship(x, z)), name="axiom-propagate-subproperty-to-class"),
    ForAll(x, y, z, Implies(And(inverseOf_relationship(x, y), propertyOf_relationship(y, z)), propertyOf_relationship(x, z)), name="axiom-propagate-inverse-to-class")
]

model.add_knowledge(*axioms)

# Query
queries = [
    Exists(x, y, subclassOf_relationship(x, y), name="query-get-all-class-subclass-relationships"),
    Exists(x, y, instanceOf_relationship(x, y), name="query-get-all-instance-class-relationships"),
    Exists(x, y, propertyOf_relationship(x, y), name="query-get-all-class-property-relationships")
]

model.add_knowledge(*queries)

# TEST: Add tailored facts to the model and test inference for subclassOf relationship
language_df_dict = language_df.to_dict('records')  # convert df to dict for faster row iteration
for row in language_df_dict:
    model.add_data({
    class_node: {row['label'].lower(): Fact.TRUE, row['supertype'].lower(): Fact.TRUE},
    subclassOf_relationship: {(row['label'].lower(), row['supertype'].lower()): Fact.TRUE}})

# TEST: practical test for english language    
model.add_data({
instance_node: {"english": Fact.TRUE},
instanceOf_relationship: {("english", "language"): Fact.TRUE}})    
    
 
# Perform inference
steps, facts_inferred = model.infer()

# Inspect the query node
for query in queries:
    print(f"Query {query.name} result:")
    print(model[query].true_groundings)

Query query-get-all-class-subclass-relationships result:
{('language', '<file:///c:/users/hanna/desktop/autonomouslearner/data/graph%20database/github_sumo#linguisticexpression>'), ('language', 'ptop:abstract')}
Query query-get-all-instance-class-relationships result:
{('english', 'language')}
Query query-get-all-class-property-relationships result:
set()


In [29]:
model.print()


***************************************************************************
                                LNN Model

OPEN Exists: query-get-all-class-property-relationships       UNKNOWN (0.0, 1.0)

OPEN Exists: query-get-all-instance-class-relationships          TRUE (1.0, 1.0)

OPEN Exists: query-get-all-class-subclass-relationships          TRUE (1.0, 1.0)

AXIOM ForAll: axiom-propagate-inverse-to-class              TRUE (1.0, 1.0)

OPEN Implies: ((inverseOf(0, 1) ∧ propertyOf(1, 2)) → propertyOf(0, 2)) 

OPEN And: (inverseOf(0, 1) ∧ propertyOf(1, 2)) 

AXIOM ForAll: axiom-propagate-subproperty-to-class          TRUE (1.0, 1.0)

OPEN Implies: ((subPropertyOf(0, 1) ∧ propertyOf(1, 2)) → propertyOf(0, 2)) 

OPEN And: (subPropertyOf(0, 1) ∧ propertyOf(1, 2)) 

AXIOM ForAll: axiom-propagate-class-property-to-instance          TRUE (1.0, 1.0)

OPEN Implies: ((Instance(0) ∧ Class(1) ∧ Property(2) ∧ instanceOf(0, 1) ∧ propertyOf(2, 1)) → propertyOf(2, 0)) 

OPEN And: (Instance(0) ∧ Clas

In [30]:
# the model learns the FOL rule for english
# we are able to reproduce the behavior from experiment 0 successfully
# now let's go a step further and insert the new knowledge into the graph
# creating the feedback loop between learning and reasoning

# now query again to see if we retrive both language concept classes in our unified knowledge graph
# query class hierarchy
sparql_insert_english_language = """
INSERT {
 ptop:english rdf:type ptop:English .
 ptop:english rdfs:subClassOf ptop:Language .
 ptop:english rdfs:label "english"
}
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig27.html")

tmp.fig27.html


In [31]:
# now query again to see if we retrive both language concept classes in our unified knowledge graph
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig260.html")

tmp.fig260.html


In [32]:
english_df = df[df["label"].str.lower() == "english"]
english_df

,subject,label,supertype


In [33]:
for nodeid in nx_graph.nodes:
    if "language" in nx_graph.nodes[nodeid]['label'].lower() or "subclass" in nx_graph.nodes[nodeid]['label'].lower():
        print(nx_graph.nodes[nodeid], nodeid)

{'label': 'ptop:Language', 'bipartite': 1} 46
{'label': 'A spoken or written natural language', 'bipartite': 1} 47
{'label': 'Language', 'bipartite': 1} 48
{'label': 'A textual description of an entity. Usually a free text in some natural language. As defined in DC2003ISO for InformationResources. In a sense, it is a specializatoin of rdf:comment', 'bipartite': 1} 94
{'label': 'ptop:inLanguage', 'bipartite': 0} 196
{'label': 'The language of the intellectual content of a resource. Recommended best practice is to use RFC 3066 [RFC3066], which, in conjunction with ISO 639 [ISO639], defines two- and three-letter primary language tags with optional subtags. Examples include “en” or “eng” for English, “akk” for Akkadian, and “en-GB” for English used in the United Kingdom. DC2003ISO', 'bipartite': 1} 197
{'label': 'in Language', 'bipartite': 1} 198


In [34]:
nodeids = list(nx_graph.nodes)
print(nodeids[-1])

323


In [35]:
list(nx_graph.nodes(data=True))

[(0, {'label': '<http://www.ontotext.com/proton/protontop>', 'bipartite': 0}),
 (1, {'label': 'Proton', 'bipartite': 1}),
 (2, {'label': 'Proton Ontology', 'bipartite': 1}),
 (3,
  {'label': 'Upper-level ontology with extensions to handle Linked Open Data (LOD)',
   'bipartite': 1}),
 (4,
  {'label': 'Ontotext (Kiril Simov, Atanas Kiryakov, Ivan Terziev, Dimitar Manov, Mariana Damova, Svetoslav Petrov)',
   'bipartite': 1}),
 (5, {'label': 'PROTON (Proto Ontology), Top module', 'bipartite': 1}),
 (6, {'label': 'owl:Ontology', 'bipartite': 1}),
 (7, {'label': '3.0', 'bipartite': 1}),
 (8,
  {'label': 'Proton ontology is available under the Creative Commons Attribution 3.0 Unported license; see http://creativecommons.org/licenses/by/3.0/. In a nutshell, you are free to copy, distribute and transmit the work; to remix/adapt the work (e.g. to import the ontology and create specializations of its elements), as long as you attribute the work in the manner specified by the author or licensor 

In [46]:
for s, p, o in kg.rdf_graph():
    if "subclass" in p.lower() and "language" in o.lower():
        print(s,p,o)

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#BalticLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#IndoEuropeanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#PareCushiticMixedLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#MixedLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#EasternIranianLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#IranianLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#YanomamLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SpokenHumanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20databas

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#ChonLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SpokenHumanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#CastilianLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WestIberianLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#TransNewGuineaLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SpokenHumanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#NorthernChukotkoKamchatkanLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#ChukotkoKamchatkanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLea

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#FrenchGroupLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#OielLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WesternRomanceLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#ItaloWesternRomanceLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#AltaicLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SpokenHumanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#KaoRiverLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#NorthernNorthHalmaheraLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WitotoGroupLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WitotoanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#BirdsHeadLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WestPapuanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#GermanBasedCreoleLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#CreoleLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SwedishGroupLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#DanishSwedishLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#NorthwesternZoneIndoAryanLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#IndoAryanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WitotoanLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SpokenHumanLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#SouthernTurkicLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#TurkicLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#TiatinaguaLanguage <class 'rdflib.term.URIRef'> file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#TacananLanguage
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/grap

In [64]:
import rdflib

# add english language predicate to knowledge graph
eng_s = rdflib.URIRef("file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#HannaEnglish")
eng_p = rdflib.URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf")
eng_o = rdflib.URIRef("file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#Language")
kg.add(eng_s, eng_p, eng_o)

In [54]:
# now query again to see if we retrive both language concept classes in our unified knowledge graph
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig261.html")

tmp.fig261.html


In [63]:
english_df = df[df["label"].str.lower() == "language"]
english_df

,subject,label,supertype
2655,<file:///C:/Users/Hanna/Desktop/AutonomousLear...,language,<file:///C:/Users/Hanna/Desktop/AutonomousLear...
5468,ptop:Language,Language,ptop:Abstract


In [75]:
# get all namespaces in knowedge graph
kg.get_ns_dict()

{'dct': 'http://purl.org/dc/terms/',
 'owl': 'http://www.w3.org/2002/07/owl#',
 'prov': 'http://www.w3.org/ns/prov#',
 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
 'schema': 'http://schema.org/',
 'sh': 'http://www.w3.org/ns/shacl#',
 'skos': 'http://www.w3.org/2004/02/skos/core#',
 'xsd': 'http://www.w3.org/2001/XMLSchema#',
 'xml': 'http://www.w3.org/XML/1998/namespace',
 'dc': 'http://purl.org/dc/elements/1.1/',
 'ptop': 'http://www.ontotext.com/proton/protontop#',
 '': 'http://www.ontologyportal.org/SUMO.owl#',
 'wn': 'http://www.ontologyportal.org/WordNet.owl#'}

In [76]:
eng_s_proton = rdflib.URIRef("http://www.ontotext.com/proton/protontop#HannaEnglish2" )
eng_p_proton = rdflib.URIRef("http://www.w3.org/2000/01/rdf-schema#subClassOf" )
eng_o_proton = rdflib.URIRef("http://www.ontotext.com/proton/protontop#Language")
kg.add(eng_s_proton, eng_p_proton, eng_o_proton)

In [77]:
# now query again to see if we retrive both language concept classes in our unified knowledge graph
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    OPTIONAL { ?subject rdfs:label ?label }
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig2611.html")

tmp.fig2611.html


In [81]:
english_df = df[df["label"].str.lower() == "http://www.ontotext.com/proton/protontop#hannaenglish2"]
english_df

,subject,label,supertype


In [90]:
for s, p, o in kg.rdf_graph():
    if "language" in s.lower() and "http://www.w3.org/2000/01/rdf-schema#label" in p.lower():
        print(s,p,o)

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#FrenchGroupLanguage http://www.w3.org/2000/01/rdf-schema#label french group language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#CayubabaLanguage http://www.w3.org/2000/01/rdf-schema#label cayubaba language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#MaritimeSignLanguage http://www.w3.org/2000/01/rdf-schema#label maritime sign language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#TurksAndCaicosCreoleEnglishLanguage http://www.w3.org/2000/01/rdf-schema#label turks and caicos creole english language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#MisumalpanLanguage http://www.w3.org/2000/01/rdf-schema#label misumalpan language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#YukiGroupLanguage http://www.w3.org/2000/01/rdf-sc

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#NafusiLanguage http://www.w3.org/2000/01/rdf-schema#label nafusi language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#EasternTurkicLanguage http://www.w3.org/2000/01/rdf-schema#label eastern turkic language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#CentralIranianLanguage http://www.w3.org/2000/01/rdf-schema#label central iranian language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WesterwoldsLanguage http://www.w3.org/2000/01/rdf-schema#label westerwolds language
http://www.ontotext.com/proton/protontop#Language http://www.w3.org/2000/01/rdf-schema#label Language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#IstroRomanianLanguage http://www.w3.org/2000/01/rdf-schema#label istro romanian language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/gr

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#JudeoGeorgianLanguage http://www.w3.org/2000/01/rdf-schema#label judeo georgian language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#MalaysianSignLanguage http://www.w3.org/2000/01/rdf-schema#label malaysian sign language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#ToraLanguage http://www.w3.org/2000/01/rdf-schema#label tora language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#JapaneseGroupLanguage http://www.w3.org/2000/01/rdf-schema#label japanese group language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#PurepechaLanguage http://www.w3.org/2000/01/rdf-schema#label purepecha language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#LaurentianLanguage http://www.w3.org/2000/01/rdf-schema#label laurentian language
fil

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#LibyanSignLanguage http://www.w3.org/2000/01/rdf-schema#label libyan sign language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#KualaLumpurSignLanguage http://www.w3.org/2000/01/rdf-schema#label kuala lumpur sign language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#Language http://www.w3.org/2000/01/rdf-schema#label language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#WaremboriLanguage http://www.w3.org/2000/01/rdf-schema#label warembori language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#PawneeKitsaiLanguage http://www.w3.org/2000/01/rdf-schema#label pawnee kitsai language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#PidginLanguage http://www.w3.org/2000/01/rdf-schema#label pidgin language
file:///C:/Users/Hanna/

file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#ZambianSignLanguage http://www.w3.org/2000/01/rdf-schema#label zambian sign language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#NorthernNambikuaraLanguage http://www.w3.org/2000/01/rdf-schema#label northern nambikuara language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#EnglishTahitianLanguage http://www.w3.org/2000/01/rdf-schema#label english tahitian language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#TurkicLanguage http://www.w3.org/2000/01/rdf-schema#label turkic language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#ItalicLanguage http://www.w3.org/2000/01/rdf-schema#label italic language
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#AmamiOkinawanLanguage http://www.w3.org/2000/01/rdf-schema#label amami okinaw

In [91]:
eng_s_proton = rdflib.URIRef("http://www.ontotext.com/proton/protontop#HannaEnglish2" )
eng_p_proton = rdflib.URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type" )
eng_o_proton = rdflib.URIRef("http://www.w3.org/2002/07/owl#Class")
kg.add(eng_s_proton, eng_p_proton, eng_o_proton)

In [104]:
# now query again to see if we retrive both language concept classes in our unified knowledge graph
# query class hierarchy
sparql = """
SELECT *
  WHERE {
    ?property ?range ?domain
  } ORDER BY ?property
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig2611.html")

tmp.fig2611.html


In [105]:
english_df = df[df["subject"].str.lower() == "http://www.ontotext.com/proton/protontop#hannaenglish2"]
english_df

,subject,label,supertype


In [108]:
for s, p, o in kg.rdf_graph():
    if "hannaenglish" in s.lower():
        print(s,p,o)

http://www.ontotext.com/proton/protontop#HannaEnglish2 http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.ontotext.com/proton/protontop#Language
http://www.ontotext.com/proton/protontop#HannaEnglish2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#HannaEnglish http://www.w3.org/2000/01/rdf-schema#subClassOf file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#Language


In [109]:
eng_s_proton = rdflib.URIRef("http://www.ontotext.com/proton/protontop#HannaEnglish2" )
eng_p_proton = rdflib.URIRef("http://www.w3.org/2000/01/rdf-schema#label" )
eng_o_proton = rdflib.URIRef("HannaEnglish2")
kg.add(eng_s_proton, eng_p_proton, eng_o_proton)

In [132]:
# query class hierarchy
sparql = """
SELECT DISTINCT ?subject ?label ?supertype
WHERE {
    { ?subject a owl:Class . } UNION { ?individual a ?subject . } .
    OPTIONAL { ?subject rdfs:subClassOf ?supertype } .
    ?subject rdfs:label 'HannaEnglish2'
} ORDER BY ?subject
  """

# visualize query
pyvis_graph = kg.visualize_query(sparql, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig06.html")

tmp.fig06.html


In [133]:
english_df = df[df["label"].str.lower() == "hannaenglish2"]
english_df

,subject,label,supertype


In [128]:
for s, p, o in kg.rdf_graph():
    if "hannaenglish" in s.lower():
        print(s,p,o)

http://www.ontotext.com/proton/protontop#HannaEnglish2 http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.ontotext.com/proton/protontop#Language
http://www.ontotext.com/proton/protontop#HannaEnglish2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#HannaEnglish http://www.w3.org/2000/01/rdf-schema#subClassOf file:///C:/Users/Hanna/Desktop/AutonomousLearner/data/graph%20database/github_sumo#Language
http://www.ontotext.com/proton/protontop#HannaEnglish2 http://www.w3.org/2000/01/rdf-schema#label HannaEnglish2
